In [128]:
!wget https://github.com/kevin900804/Operation_Research/raw/main/data.xlsx

--2022-08-10 09:45:15--  https://github.com/kevin900804/Operation_Research/raw/main/data.xlsx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kevin900804/Operation_Research/main/data.xlsx [following]
--2022-08-10 09:45:15--  https://raw.githubusercontent.com/kevin900804/Operation_Research/main/data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13242 (13K) [application/octet-stream]
Saving to: ‘data.xlsx.5’

data.xlsx.5         100%[===================>]  12.93K  --.-KB/s    in 0s      

2022-08-10 09:45:15 (94.8 MB/s) - ‘data.xlsx.5’ saved [13242/13242]



In [129]:
import pandas as pd
problem2 = pd.read_excel("/content/data.xlsx", "Problem 2")
problem2.drop(columns = problem2.columns[10:], inplace = True)
problem2.columns = ['District (from)', 1,2,3,4,5,6,7,8,'Population']

districts = problem2['District (from)'].iloc[1:]
distance = problem2.iloc[1:, 1:9]
population = problem2['Population'].iloc[1:]
ambulanceNum = 2

In [130]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com, https://us-python.pkg.dev/colab-wheels/public/simple/


In [131]:
from gurobipy import *
firefighting = Model("firefighting")

w = {} # the distance between District ii and its closest ambulance
x = {} # is 1 if an ambulance is located in District j and 0 otherwise
y = {} # is 1 if for District i the closest ambulance is located in District j and 0 otherwise
for i in districts:
  w[i] = firefighting.addVar(lb = 0, vtype = GRB.INTEGER, name = "w" + str(i))
  x[i] = firefighting.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(i))
  for j in districts:
    y[i, j] = firefighting.addVar(lb = 0, vtype = GRB.BINARY, name = "y" + str(i) + str(j))

z = firefighting.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "object value")

firefighting.setObjective(z, GRB.MINIMIZE)

firefighting.addConstr(quicksum(x[i] for i in districts) == ambulanceNum)
firefighting.addConstrs(y[i, j] <= x[i] for i in districts for j in districts)
firefighting.addConstrs(quicksum(y[i, j] for i in districts) == 1 for j in districts)
firefighting.addConstrs((w[i] >= quicksum(y[j, i] * distance[j][i] for j in districts) for i in districts))
firefighting.addConstrs(z >= w[i] * population[i] for i in districts)

firefighting.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 89 rows, 81 columns and 280 nonzeros
Model fingerprint: 0x65f0d839
Variable types: 1 continuous, 80 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 240.0000000
Presolve removed 22 rows and 22 columns
Presolve time: 0.00s
Presolved: 67 rows, 59 columns, 208 nonzeros
Variable types: 0 continuous, 59 integer (58 binary)

Root relaxation: objective 8.289474e+01, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   82.89474    0   32  240.00000   82.89474  65.5%     -    0s
H    0     0                     180.000000

In [132]:
print("The ambulance should be located in:", end = " ")
for i in districts:
  if x[i].x == 1:
    print("District" + str(int(i)), end = " ")
print()

print("The population-weighted firefighting time:")
for i in districts:
  print("District" + str(int(i)) + ":", w[i].x * population[i])

print("The maximum population-weighted firefighting time =", firefighting.objVal)

The ambulance should be located in: District1 District6 
The population-weighted firefighting time:
District1: 0.0
District2: 90.0
District3: 105.0
District4: 120.0
District5: 120.0
District6: 0.0
District7: 135.0
District8: 120.0
The maximum population-weighted firefighting time = 135.0
